In [ ]:
#%pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 1.3 MB/s eta 0:00:00a 0:00:01
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.9/443.9 kB 2.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 2.8 MB/s eta 0:00:00a 0:00:01
Using cached click-8.1.8-py3-none-any.whl (98 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 3.4 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: python-dateutil
    Found existing installation: python-dateutil 2.8.1
    Uninstalling python-dateutil-2.8.1:
      Successfully uninstalled python-dateutil-2.8.1
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.33.1
    Uninstalling pydantic_core-2.33.1:
      Successfully uninstalled pydantic_core-2.33.1
  Attempting uninstall: click
    Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2
  Attempting unin

In [ ]:
from datetime import datetime
import os
import gradio as gr

from nClient import (
    create_socket,
    connect_socket,
    send_request,
    handle_response_content,
    save_response_content,
)

# ---------------------------
# HTTP CLIENT BACK‑END LOGIC
# ---------------------------

def _clean_host_and_port(raw_host: str, raw_port: str):
    """Normaliza host (sin protocolo / path extra) y puerto."""
    host = (raw_host or "localhost").strip()
    # Elimina http(s)://
    for prefix in ("http://", "https://"):
        if host.startswith(prefix):
            host = host[len(prefix):]
            break
    # Si vino con path (foo.com/abc) lo cortamos
    if "/" in host:
        host = host.split("/", 1)[0]

    port = int(raw_port) if raw_port and raw_port.isdigit() else 80
    return host, port


def _parse_custom_headers(headers_raw: str):
    """Convierte el textbox multilinea en lista de cabeceras 'Header: valor'."""
    if not headers_raw:
        return []
    return [h.strip() for h in headers_raw.splitlines() if h.strip()]


def _bytes_len(data):
    return len(data if isinstance(data, (bytes, bytearray)) else data.encode("utf-8"))


def client_request(
    host,
    port,
    method,
    path,
    save_response,
    save_filename,
    headers_raw,
    upload_local_path,
    manual_body,
    upload_flag,
):
    """Envia la petición y devuelve la respuesta como string legible."""

    host, port = _clean_host_and_port(host, port)
    method = (method or "GET").upper().strip()
    if method not in {"GET", "POST", "PUT", "DELETE", "HEAD"}:
        return "Método HTTP no válido."

    # ————————————————————————
    # Preparación cabeceras y cuerpo
    # ————————————————————————
    custom_headers = _parse_custom_headers(headers_raw)

    is_request_binary = False  # Solo afecta al cuerpo que ENVIAMOS
    body_bytes = b""
    content_type = "text/plain"

    if method in {"POST", "PUT"}:
        if upload_flag and upload_local_path:
            # 1️⃣  Subida de fichero desde disco
            local_path = upload_local_path  # Ruta en disco
            ext = os.path.splitext(local_path)[1].lower()[1:]
            is_request_binary = ext in {"png", "jpg", "jpeg", "gif", "mp3", "wav", "mp4", "avi"}

            # Content‑Type según extensión
            ct_map = {
                "jpg": "image/jpeg",
                "jpeg": "image/jpeg",
                "png": "image/png",
                "gif": "image/gif",
                "mp3": "audio/mpeg",
                "wav": "audio/wav",
            }
            content_type = ct_map.get(ext, "application/octet-stream" if is_request_binary else "text/plain")

            mode = "rb" if is_request_binary else "r"
            with open(local_path, mode) as f:
                body_bytes = f.read() if is_request_binary else f.read().encode("utf-8")
        else:
            # 2️⃣  Cuerpo manual (JSON generalmente)
            body_bytes = (manual_body or "").encode("utf-8")
            content_type = "application/json"
    elif method in {"DELETE", "HEAD"}:
        body_bytes = b""  # sin cuerpo
    else:  # GET
        body_bytes = b""

    # ————————————————————————
    # Construir request 100 % especificación
    # ————————————————————————
    request_lines = [
        f"{method} {path} HTTP/1.1",
        f"Host: {host}",
        "Connection: close",
    ]

    if method in {"POST", "PUT"}:
        request_lines.append(f"Content-Type: {content_type}")
        request_lines.append(f"Content-Length: {_bytes_len(body_bytes)}")

    # custom headers
    request_lines.extend(custom_headers)
    request_lines.append("")  # línea vacía fin cabeceras

    request_message = "\r\n".join(request_lines).encode("utf-8") + b"\r\n" + body_bytes

    # ————————————————————————
    # Enviar y recibir
    # ————————————————————————
    sock = create_socket()
    connect_socket(sock, host, port)
    response_raw = send_request(sock, request_message, is_binary=False)  # siempre esperamos texto en la respuesta de control
    sock.close()

    # `send_request` ya devuelve str (porque is_binary=False). Garantizamos string.
    if isinstance(response_raw, (bytes, bytearray)):
        response_text = response_raw.decode("utf-8", errors="replace")
    else:
        response_text = response_raw

    # Limpieza visual: sustituir CRLF por LF para que se vea bonito en la textbox
    response_text = response_text.replace("\r\n", "\n")

    # Guardar a disco si es GET y el usuario lo pidió
    if method == "GET" and save_response:
        headers_block, ctype, payload = handle_response_content(response_raw, False)
        is_binary_download = ctype and any(x in ctype.lower() for x in ("image/", "audio/", "video/", "application/octet-stream"))
        if payload:
            save_response_content(payload, save_filename or "download.bin", is_binary_download)

    return response_text

# ---------------------------
#  UI HELPERS
# ---------------------------

def update_ui(selected_method):
    """Muestra/oculta controles según el método HTTP elegido."""
    is_body_method = selected_method in {"POST", "PUT"}
    return (
        gr.update(visible=not is_body_method),  # save_response checkbox
        gr.update(visible=is_body_method),      # upload_file_checkbox
        gr.update(visible=False),               # upload_file (inicialmente oculto; lo mostrará el checkbox)
        gr.update(visible=is_body_method),      # manual_body
    )


def toggle_upload_controls(use_file):
    """Cuando el usuario marca 'Upload file', se oculta el textarea y aparece el selector de archivo."""
    return (
        gr.update(visible=use_file),   # upload_file
        gr.update(visible=not use_file),  # manual_body
    )


def _get_file_path(file_obj):
    return file_obj.name if file_obj else ""

# ---------------------------
#  GRADIO INTERFACE
# ---------------------------
with gr.Blocks() as iface:
    with gr.Row():
        host_tb = gr.Textbox(label="Host", value="localhost")
        port_tb = gr.Textbox(label="Port", value="8080")

    method_rb = gr.Radio(
        choices=["GET", "POST", "PUT", "DELETE", "HEAD"],
        label="HTTP Method",
        value="GET",
    )

    path_tb = gr.Textbox(label="Path", value="/")

    save_cb = gr.Checkbox(label="Save file (GET)", value=False)
    save_name_tb = gr.Textbox(label="Filename", visible=False)
    save_cb.change(lambda x: gr.update(visible=x), inputs=save_cb, outputs=save_name_tb)

    headers_tb = gr.Textbox(label="Custom Headers", lines=3, placeholder="X-Custom: 123")

    upload_file_cb = gr.Checkbox(label="Upload file", visible=False)
    manual_body_tb = gr.Textbox(label="Body", lines=8, visible=False)
    upload_file_input = gr.File(label="Choose file", type="filepath", visible=False)

    # keep file path hidden (Gradio File returns tempfile path)
    file_path_hidden = gr.Textbox(visible=False)
    upload_file_input.change(_get_file_path, inputs=upload_file_input, outputs=file_path_hidden)

    # Toggle controls when checkbox marked
    upload_file_cb.change(
        toggle_upload_controls,
        inputs=upload_file_cb,
        outputs=[upload_file_input, manual_body_tb],
    )

    # Change UI when method switches
    method_rb.change(
        update_ui,
        inputs=method_rb,
        outputs=[save_cb, upload_file_cb, upload_file_input, manual_body_tb],
    )

    response_tb = gr.Textbox(label="Raw HTTP Response", lines=15)
    send_btn = gr.Button("Send request ✈️")

    send_btn.click(
        fn=client_request,
        inputs=[
            host_tb,
            port_tb,
            method_rb,
            path_tb,
            save_cb,
            save_name_tb,
            headers_tb,
            file_path_hidden,
            manual_body_tb,
            upload_file_cb,
        ],
        outputs=response_tb,
    )

if __name__ == "__main__":
    iface.launch(debug=True)


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Connected (plain) to localhost:8080
Keyboard interruption in main thread... closing server.
